<a href="https://colab.research.google.com/github/PETEROA/Model-Compression-Transfer_Learning/blob/main/Distill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets transformers accelerate peft

import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from transformers import AutoModel
from huggingface_hub import login
from torch.nn.utils.rnn import pad_sequence
from collections.abc import Mapping
import torch
import numpy as np


In [3]:
# Login to Hugging Face Hub
login()


The LLaMa 2 model serves as the teacher model in the distillation process. The AutoModelForCausallLM.from_pretrained function loads the model into memory. Also considers memory usage reduction and boosting inference speeds (lower-precision floating-point numbers)

In [4]:
# Load LLaMA 2 model and Tokenizer
teacher_model_name = "meta-llama/Llama-2-7b-hf"
teacher_model = AutoModelForCausalLM.from_pretrained(
    teacher_model_name, torch_dtype=torch.float16, use_auth_token=True
)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

This loads the tokenizer associated with the LLaMa model, crucial for the conversion of raw text into tokenized inputs. The AutoTokenizer.from_pretrained function is used to get the tokenizer from the hugging face repository.

In [5]:
 # Added use_auth_token=True
teacher_tokenizer = AutoTokenizer.from_pretrained(
    teacher_model_name, use_auth_token=True
)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

We load a smaller student model to be trained with knowledge distillation from the larger model. We utilise DistilBert(distilbert-base-uncased) a general pupose lightweight transformer primarily designed for efficiency.

In [6]:
# Load Smaller student model (DistilBERT-based transformer in this instance)
student_model_name = "distilbert-base-uncased"
# Use AutoModel instead of AutoModelForCausalLM for DistilBERT
student_model = AutoModel.from_pretrained(student_model_name, torch_dtype=torch.float16)
student_tokenizer = AutoTokenizer.from_pretrained(student_model_name)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

A subset of the OpenAssistant Conversations Dataset is used for training.


In [8]:
# Load dataset
dataset = load_dataset("openassistant/oasst1", split="train[:5%]")


We Tokenize the input text by utilizing the student model's tokenizer. Padding and truncation are handled. Also carry out conversion into PyTorch tensors for model compatibility.

In [9]:
# Tokenization function
def tokenize_function(examples):
    # Add a padding token to the student_tokenizer if it doesn't have one
    if student_tokenizer.pad_token is None:
        student_tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    # Return a dictionary with both input_ids and attention_mask
    tokenized_output = student_tokenizer(
        examples["text"],
        padding="max_length",
        max_length=512,
        truncation=True,
        return_tensors="pt",  # returns PyTorch tensors
    )

This function tokenizes data for training the student model. The goal is to ensure appropraite input formatting and label assignment.


In [10]:
# Tokenization function
def tokenize_function(examples):
    # Add a padding token to the student_tokenizer if it doesn't have one
    if student_tokenizer.pad_token is None:
        student_tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    # Use the student tokenizer instead of the teacher tokenizer
    # Return a dictionary with both input_ids and attention_mask
    tokenized_output = student_tokenizer(
        examples["text"],
        padding="max_length",
        max_length=512,
        truncation=True,
        return_tensors="pt",
    )
    # Add a 'labels' key with the same value as 'input_ids'
    # Ensure 'input_ids' exists and is not None before assigning it to 'labels'
    if "input_ids" in tokenized_output and tokenized_output["input_ids"] is not None:
        tokenized_output["labels"] = tokenized_output["input_ids"]
    else:
        # Handle the case where 'input_ids' is missing or None
        # For example, you can assign a default value or skip the example
        tokenized_output["labels"] = torch.empty(
            0, dtype=torch.long
        )  # Empty tensor

    return tokenized_output

In [11]:
# Apply the tokenization function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/4222 [00:00<?, ? examples/s]

In [12]:
# Knowledge Distillation Loss
class Distillation_loss(nn.Module):
    def __init__(self, alpha=0.5, temperature=2.0):
        super(Distillation_loss, self).__init__()
        self.alpha = alpha
        self.temperature = temperature
        self.kl_div = nn.KLDivLoss(reduction="batchmean")

    def forward(self, student_logits, teacher_logits, labels):
        hardloss = nn.CrossEntropyLoss()(student_logits, labels)
        softloss = self.kl_div(
            nn.functional.log_softmax(student_logits / self.temperature, dim=-1),
            nn.functional.softmax(teacher_logits / self.temperature, dim=-1),
        )
        return self.alpha * hardloss + (1 - self.alpha) * softloss



In [13]:
# Apply LoRA for efficient Fine-Tunning
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"],  # target_modules to match DistilBERT
    lora_dropout=0.05,
    bias="none",
)
student_model = get_peft_model(student_model, lora_config)


In [14]:
# Update TrainingArguments to remove unused columns
training_args = TrainingArguments(
    output_dir="./distilled_llm",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_steps=100,
    save_strategy="epoch",
    evaluation_strategy="no",
    fp16=True,
    remove_unused_columns=False,
)



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
# data collator
def torch_default_data_collator(features):
    """
    Very simple data collator that:
    - simply collates batches of dict-like objects
    - Performs special handling for potential keys named:
        - `label`: handles a single value (int or float) per object
        - `label_ids`: handles a list of values per object
    - Handles a single value per object for all other keys
    - Handles None values by replacing them with 0
    """
    first = features[0]
    batch = {}

    if "label" in first and type(first["label"]) is not list:
        labels = [f["label"] for f in features]
        try:
            labels = [0 if label is None else label for label in labels]
            batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        except ValueError:
            batch["labels"] = torch.empty(0, dtype=torch.long)
    elif "label_ids" in first and type(first["label_ids"]) is list:
        if all(l is None or x is not None and type(x) in {int, float} for f in features for x in f["label_ids"] for l in features):  # noqa
            label_ids = [f["label_ids"] for f in features]
            try:
                label_ids = [[0 if x is None else x for x in ids] for ids in label_ids]
                padding_value = features[0].get("padding_value", 0)
                batch["label_ids"] = pad_sequence([torch.tensor(l) for l in label_ids], batch_first=True, padding_value=padding_value)  # noqa
            except ValueError:
                # Handle the case where all label_ids are None
                batch["label_ids"] = torch.empty(0, dtype=torch.long)  # Empty tensor
        else:
            raise ValueError("A batch was collated with `label_ids` that are not all lists of integers or floats.")  # noqa
    # Handling of all other possible keys.
    # Again, we will use the first element to figure out which key/values are not None for this model.
    for k in first.keys():
        if k not in ("label", "label_ids") and type(first[k]) is not list and first[k] is not None:  # noqa
            try:
                # Replace None with a default value, e.g., 0
                # Filter out None values before creating the tensor
                batched = [feature.get(k, 0) for feature in features if feature.get(k, 0) is not None # Filter out None values before creating the tensor # noqa
                           and not isinstance(feature.get(k, 0), (str, bytes, dict))]  # Filter out string, byte, and dict types # noqa
                # Check if 'batched' is empty before creating the tensor
                if batched:
                    batch[k] = torch.tensor(batched)
                else:
                    # Handle empty 'batched' (e.g., skip, assign a default value)
                    pass  # You can skip the feature or assign a default value if needed # noqa
            except (ValueError, TypeError):
                # Handle cases where the data type is not supported by torch.tensor
                # You might need to skip these features or implement specific handling
                # For example, handle strings by converting them to numerical representations
                pass # noqa

    # Explicitly include 'input_ids' in the batch
    batch["input_ids"] = torch.tensor([f["input_ids"] for f in features])


    return batch

In [16]:
# Define Trainer with custom data collator and loss function
trainer = Trainer(
    model=student_model,
    args=training_args,
    train_dataset=tokenized_datasets,
    compute_metrics=None,
    data_collator=torch_default_data_collator,  # Use custom data collator
)


In [17]:
# Training loop with distillation loss
def train_with_distillation(trainer, teacher_model, student_model, tokenized_datasets):
    """Trains the student model using distillation from the teacher model."""

    distillation_loss = Distillation_loss()  # Instantiate your distillation loss

    for epoch in range(trainer.args.num_train_epochs):
        for batch in trainer.get_train_dataloader():
            # Get teacher logits
            with torch.no_grad():
                teacher_outputs = teacher_model(**batch)  # Get teacher outputs
                teacher_logits = teacher_outputs.logits  # Extract teacher logits

            # Get student logits
            student_outputs = student_model(**batch)  # Get student outputs
            student_logits = student_outputs.logits  # Extract student logits

            # Calculate and apply distillation loss
            loss = distillation_loss(
                student_logits, teacher_logits, batch["labels"]
            )  # Pass labels to the loss
            loss.backward()

            trainer.optimizer.step()  # Update student model weights
            trainer.optimizer.zero_grad()  # Reset gradients

        trainer.save_model()  # Save model at the end of each epoch

In [ ]:
# Start training
train_with_distillation(trainer, teacher_model, student_model, tokenized_datasets)